In [1]:
import numpy as np
import pandas as pd
import torch
torch.set_default_tensor_type(torch.FloatTensor) 
import copy
import sys
import os
notebook_dir = os.getcwd()
parent_dir = os.path.dirname(notebook_dir)
# Add the parent directory to the Python path
sys.path.append(parent_dir)

# sys.path.append('/Users/lucile/causal_info_gain/causal_prospective_merge/data')
# sys.path.append('/Users/lucile/causal_info_gain/causal_prospective_merge/data')
from rct_data_generator import *
from outcome_models import *
from plotting_functions import *
from mcmc_bayes_update import *
from eig_comp_utils import *
from research_exp_utils import *

# from econml.metalearners import TLearner
# from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier


/Users/jake/Python/causal_prospective_merge/.conda/lib/python3.11/site-packages/torch/__init__.py:747: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1712608632396/work/torch/csrc/tensor/python_tensor.cpp:433.)
  _C._set_default_tensor_type(t)
 /Users/jake/Python/causal_prospective_merge/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning:IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
notebook_dir = os.getcwd()

In [3]:
# from scipy.stats import kendalltau, spearmanr

# def average_precision_at_k(true_rankings, predicted_rankings, k):
#     num_hits = 0
#     sum_precision = 0
#     for i, pred in enumerate(predicted_rankings[:k], 1):
#         if pred in true_rankings:
#             num_hits += 1
#             sum_precision += num_hits / i
#     if not true_rankings:
#         return 0
#     return sum_precision / min(len(true_rankings), k)

# def mean_average_precision(true_rankings, predicted_rankings, k=None):
#     if k is None:
#         k = len(true_rankings)
#     avg_precision = np.mean([average_precision_at_k(true_rankings, predicted_rankings, k_) for k_ in range(1, k + 1)])
#     return avg_precision

# def precision_at_k(true_rankings, predicted_rankings, k):
#     intersection = set(predicted_rankings[:k]) & set(true_rankings[:k])
#     return len(intersection) / k

# def recall_at_k(true_rankings, predicted_rankings, k):
#     intersection = set(predicted_rankings[:k]) & set(true_rankings[:k])
#     return len(intersection) / len(true_rankings)

# def mrr(true_rankings, predicted_rankings):
#     for i, pred in enumerate(predicted_rankings, 1):
#         if pred in true_rankings:
#             return 1 / i
#     return 0

# def ndcg(true_rankings, predicted_rankings, k=None):
#     if k is None:
#         k = len(true_rankings)
#     dcg = sum(2 ** true_rankings[i] - 1 / np.log2(i + 2) for i in range(k))
#     ideal_rankings = sorted(true_rankings, reverse=True)
#     ideal_dcg = sum(2 ** ideal_rankings[i] - 1 / np.log2(i + 2) for i in range(k))
#     return dcg / ideal_dcg

# def compare_to_ground_truth(results_dict, true_cate_ranking, eig_ranking, merged_mse, top_n = None, k = None):
    
#     if top_n is not None:
#         topn_eig_ranking = eig_ranking[:top_n]
#         topn_true_cate_ranking = true_cate_ranking[:top_n]
#         topn_merged_mse = merged_mse[:top_n]
#     else: 
#         topn_eig_ranking, topn_true_cate_ranking,topn_merged_mse = eig_ranking, true_cate_ranking,merged_mse

#     if k is None:
#         k = len(true_cate_ranking)
    
#     implied_ranking = [eig_ranking.index(val) for val in list(range(min(eig_ranking),max(eig_ranking)+1))]
    
#     results_dict['tau'] = results_dict.get('tau',[])+[(kendalltau(implied_ranking, merged_mse)[0])]    
#     results_dict['rho'] = results_dict.get('rho',[])+[(spearmanr(implied_ranking, merged_mse)[0])]  

#     if type(k) == int:
#         results_dict['precision_at_k'] = results_dict.get('precision_at_k',[]) + [precision_at_k(true_cate_ranking, topn_eig_ranking, k=k)]
#     else:
#         for val in k:
#             results_dict['precision_at_'+str(val)] = results_dict.get('precision_at_'+str(val),[]) + [precision_at_k(true_cate_ranking, topn_eig_ranking, k=val)]
#     # results_dict['recall_at_k'].append(recall_at_k(true_cate_ranking, topn_eig_ranking, k=k[0]))
#     # if type(k) == int:
#     #     results_dict['recall_at_k'].append(recall_at_k(true_cate_ranking, topn_eig_ranking, k=k))
#     # else:
#     #     for val in k:
#     #         results_dict['recall_at_'+str(val)].append(recall_at_k(true_cate_ranking, topn_eig_ranking, k=val))
#     # results_dict['mean average precision'].append(mean_average_precision(topn_true_cate_ranking, topn_eig_ranking, k=k[0]))
#     # results_dict['ndcg'].append(ndcg(topn_true_cate_ranking, topn_eig_ranking, k[0]))
#     # results_dict['rank corr eig'].append(np.corrcoef(topn_true_cate_ranking, topn_eig_ranking)[0, 1])
#     # results_dict['mean reciprocal rank'].append(mrr(topn_true_cate_ranking, topn_eig_ranking))

#     return results_dict

In [4]:
path = "../"
data_with_groundtruth, x, t, y = get_data('twins', path)
data_with_groundtruth.dropna(inplace=True)
data_with_groundtruth = data_with_groundtruth.rename(columns={'t': 'T', 'y': 'Y'})
XandT = data_with_groundtruth.drop(columns=['Y','y0','y1','ite'])
XandT.head()

,eclamp,gestatcat1,gestatcat2,gestatcat3,gestatcat4,gestatcat5,gestatcat6,gestatcat7,gestatcat8,gestatcat9,...,brstate_reg,feduc6,dfageq,nprevistq,data_year,crace,birmon,dtotord_min,dlivord_min,T
0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,...,5.0,2.0,1.0,0.0,0.0,0.0,0.0,3.0,3.0,1.0
1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,5.0,5.0,8.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,5.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,5.0,4.0,6.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0
4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,4.0,7.0,0.0,0.0,1.0,0.0,3.0,3.0,0.0


In [5]:
number_of_candidate_sites = 10

min_sample_size_cand = 150
max_sample_size_cand = 300
host_sample_size = 400 
desired_initial_sample_size = 10**4
XandT = XandT.sample(n=desired_initial_sample_size, replace=True, random_state=42)
added_T_coef = 50 # to increase importance of T

outcome_function = None
std_true_y = 1
power_x = 1
power_x_t = 1
sigma_rand_error = 1
true_beta_great_0_prop = 0.8

exp_parameters = {'number_of_candidate_sites': number_of_candidate_sites+1, 'min_sample_size_cand': min_sample_size_cand, \
                'max_sample_size_cand': max_sample_size_cand, 'host_sample_size': host_sample_size, 'outcome_function': outcome_function, \
                'std_true_y': std_true_y, 'power_x': power_x, 'power_x_t': power_x_t}

causal_param_first_index = power_x*np.shape(XandT)[1]

In [6]:
def generating_random_sites_from(data, exp_parameters, added_T_coef=1):
    
    candidates = {}
    sample_size, number_covariates = np.shape(data)[0], np.shape(data)[1]
    function_indices = {0: lambda X: np.log(X+1), 1: lambda X: X**3, 2: lambda X: X, 3: lambda X: X**2 }
    number_of_candidate_sites = exp_parameters['number_of_candidate_sites']
    min_sample_size_cand = exp_parameters['min_sample_size_cand']
    max_sample_size_cand = exp_parameters['max_sample_size_cand']
    outcome_function = None
    std_true_y = exp_parameters['std_true_y']
    power_x = exp_parameters['power_x']
    power_x_t = exp_parameters['power_x_t']
    number_features = number_covariates
    created_sites = 0
    
    while created_sites < number_of_candidate_sites:

        np.random.seed(np.random.randint(10000))
        
        selected_features_for_subsampling = np.random.randint(2, size = number_features) 
        # binary bool vector representing selection for being an input of the sampling function
        random_coefs = [np.random.uniform(-10, 10) for _ in range(number_features)] 
        random_fct_idx = [np.random.randint(0, len(function_indices.keys())) for _ in range(number_features)] 
        
        def p_assigned_to_site(X, T, eps):
            result = 0
            for j in range(number_features-1):
                result += selected_features_for_subsampling[j] * random_coefs[j] * function_indices[random_fct_idx[j]](X[j])
            # here i use added_T_coef * random_coefs to increase importance of T
            result +=  added_T_coef * random_coefs[-1] *  function_indices[random_fct_idx[-1]](T) #selected_features_for_subsampling[-1]
            return sigmoid(result + eps)
        
        sample_size = np.random.randint(min_sample_size_cand, max_sample_size_cand + 1)  # Add 1 to include max_sample_size_cand

        if created_sites==0:
            sample_size = exp_parameters['host_sample_size']
        design_data_cand = subsample_one_dataset(XandT, p_assigned_to_site, sample_size, power_x, power_x_t, outcome_function, std_true_y, seed=np.random.randint(10000))
        design_data_cand = design_data_cand.dropna()
        any_nan = design_data_cand.isna().any().any()
        if not design_data_cand.empty and not any_nan: # we're appending
            candidates[created_sites] = design_data_cand
        else:
            number_of_candidate_sites+=1 # not appending
        created_sites += 1

    return candidates

In [7]:
#dictionnary of random sites
candidate_sites = generating_random_sites_from(XandT, exp_parameters, added_T_coef=50)

for i, cand in candidate_sites.items():
    candidate_sites[i] = pd.concat([cand, data_with_groundtruth.loc[cand.index, 'Y']], axis=1)

 /var/folders/v1/x4lr48v53397r9xsq4q70mvw0000gn/T/ipykernel_10592/2089860497.py:28: FutureWarning:Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
 /var/folders/v1/x4lr48v53397r9xsq4q70mvw0000gn/T/ipykernel_10592/2089860497.py:28: FutureWarning:Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
 /var/folders/v1/x4lr48v53397r9xsq4q70mvw0000gn/T/ipykernel_10592/2089860497.py:28: FutureWarning:Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
 /var/folders/v1/x4lr48v53397r9xsq4q70mvw0000gn/T/ipyke

In [8]:
beta = (np.random.randn(152) > true_beta_great_0_prop)
beta = beta - np.mean(beta)

for i, cand in candidate_sites.items():
    candidate_sites[i]["Y"] = candidate_sites[i].drop(columns=["Y"]) @ beta 
    candidate_sites[i]["Y"] += 1 * np.random.randn(len(candidate_sites[i]["Y"]))
    

In [9]:
host = candidate_sites[0]
candidate_sites = {key: value for key, value in candidate_sites.items() if key != 0}
XandT_host, Y_host = torch.from_numpy(host.drop(columns=["Y"]).values), torch.from_numpy(host["Y"].values)

# Prior parameters for Bayesian update on host
d = np.shape(host)[1]-1
prior_mean = torch.zeros(d)
sigma_prior = 1
beta_0, sigma_0_sq, inv_cov_0 = prior_mean, sigma_rand_error,torch.eye(d)
prior_hyperparameters = {'beta_0': beta_0, 'sigma_0_sq': sigma_0_sq,"inv_cov_0":inv_cov_0}

In [10]:
n_samples_outer_expectation_obs = 400
n_samples_inner_expectation_obs = 800
n_samples_outer_expectation_caus = 400
n_samples_inner_expectation_caus = 800

sampling_parameters = {'n_samples_inner_expectation_obs':n_samples_inner_expectation_obs, 'n_samples_outer_expectation_obs':n_samples_outer_expectation_obs, \
                       'n_samples_inner_expectation_caus':n_samples_inner_expectation_caus, 'n_samples_outer_expectation_caus':n_samples_outer_expectation_caus}

eig_results = {"EIG_obs_from_samples": [], 'EIG_caus_from_samples':[], "EIG_obs_closed_form":[], "EIG_caus_closed_form":[], "EIG_obs_bart":[], "EIG_caus_bart":[]}

In [11]:
print(f" % treated in host: {round(100 * host['T'].mean(),2)}%")

 % treated in host: 74.0%


In [12]:
for _,candidate in candidate_sites.items():
    print(f"For a sample size of {np.shape(candidate)[0]}")
    print(f" % treated in candidate: {round(100 * candidate['T'].mean(),2)}%")

For a sample size of 207
 % treated in candidate: 82.13%
For a sample size of 170
 % treated in candidate: 83.53%
For a sample size of 289
 % treated in candidate: 46.71%
For a sample size of 297
 % treated in candidate: 95.96%
For a sample size of 166
 % treated in candidate: 56.63%
For a sample size of 244
 % treated in candidate: 57.79%
For a sample size of 229
 % treated in candidate: 100.0%
For a sample size of 152
 % treated in candidate: 62.5%
For a sample size of 259
 % treated in candidate: 86.49%
For a sample size of 299
 % treated in candidate: 77.93%


In [13]:
for _, candidate in candidate_sites.items():
    X_cand = torch.from_numpy(candidate.drop(columns=["Y"]).values)
    bayes_reg = BayesianLinearRegression(prior_hyperparameters)
    bayes_reg.set_causal_index(causal_param_first_index)
    post_host_parameters = bayes_reg.fit(XandT_host, Y_host)
    n_samples = n_samples_outer_expectation_obs * (n_samples_inner_expectation_obs + 1)

    eig_results["EIG_obs_closed_form"].append(
            bayes_reg.closed_form_obs_EIG(X_cand)
            )
    eig_results["EIG_caus_closed_form"].append(
            bayes_reg.closed_form_causal_EIG(X_cand)
            )

In [14]:
# eig_results["EIG_obs_from_samples"]=[]
# eig_results["EIG_caus_from_samples"]=[]

# for i, candidate in candidate_sites.items():
#     print("from samples "+str(i))
#     X_cand = torch.from_numpy(candidate.drop(columns=["Y"]).values)
#     bayes_reg = BayesianLinearRegression(prior_hyperparameters)
#     bayes_reg.set_causal_index(causal_param_first_index)
#     post_host_parameters = bayes_reg.fit(XandT_host, Y_host)

#     eig_results["EIG_obs_from_samples"].append(
#             bayes_reg.samples_obs_EIG(
#                 X_cand, n_samples_outer_expectation_obs, n_samples_inner_expectation_obs
#             )
#         )
#     eig_results["EIG_caus_from_samples"].append(
#             bayes_reg.samples_causal_EIG(
#                 X_cand, n_samples_outer_expectation_obs, n_samples_inner_expectation_obs
#             )
#         )

In [15]:
# now merge and compute some CATE error
merged_datasets = {}

for i, candidate in candidate_sites.items():
    merged_datasets[i]= pd.concat([host, candidate], axis=0)

In [16]:
cate_diff = {}
merged_mse = []
XandT_host=host.drop(columns=["Y"])

X_zero = XandT_host.copy() # we predict on host with T=0 and T=1
X_zero.iloc[:,causal_param_first_index:] = 0

X_one = XandT_host.copy()
X_one.iloc[:,causal_param_first_index:] = XandT_host.iloc[:,:causal_param_first_index]

### Merging and computing ground truth

In [17]:
data_with_groundtruth

,eclamp,gestatcat1,gestatcat2,gestatcat3,gestatcat4,gestatcat5,gestatcat6,gestatcat7,gestatcat8,gestatcat9,...,data_year,crace,birmon,dtotord_min,dlivord_min,T,Y,y0,y1,ite
0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,3.0,3.0,1.0,0.0,1.0,0.0,-1.0
1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,-1.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,3.0,3.0,0.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11979,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,10.0,6.0,2.0,1.0,0.0,0.0,0.0,0.0
11980,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,11.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
11981,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,2.0,0.0,11.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0
11982,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,2.0,0.0,11.0,3.0,3.0,0.0,0.0,0.0,1.0,1.0


In [18]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.metrics import log_loss

merged_mse = []

for i, candidate in merged_datasets.items():

    XandT_merged = candidate.drop(columns=["Y"])
    Y_merged = candidate['Y']

    learner = Ridge(fit_intercept=True)
    learner.fit(y=Y_merged, X=XandT_merged) # we fit on merged datasets

    true_cate = (X_one - X_zero) @ beta

    pred_cate = learner.predict(X_one)-learner.predict(X_zero)

    merged_mse.append(mean_squared_error(true_cate, pred_cate))


In [19]:
learner.coef_.max()

1.2139867053950197

### Comparing our EIGs with ground truth

In [20]:
obs_eig_ranking_closed_form = sorted(range(len(eig_results["EIG_obs_closed_form"])), key=lambda i: eig_results["EIG_obs_closed_form"][i], reverse=True)
print(obs_eig_ranking_closed_form)

caus_eig_ranking_closed_form = sorted(range(len(eig_results["EIG_caus_closed_form"])), key=lambda i: eig_results["EIG_caus_closed_form"][i], reverse=True)
print(caus_eig_ranking_closed_form)

# obs_eig_ranking_from_samples = sorted(range(len(eig_results["EIG_obs_from_samples"])), key=lambda i: eig_results["EIG_obs_from_samples"][i], reverse=True)
# print(obs_eig_ranking_from_samples)

# caus_eig_ranking_from_samples = sorted(range(len(eig_results["EIG_caus_from_samples"])), key=lambda i: eig_results["EIG_caus_from_samples"][i], reverse=True)
# print(caus_eig_ranking_from_samples)

true_cate_ranking = sorted(range(len(merged_mse)), key=lambda i: merged_mse[i], reverse=False) # reverse is False because its error terms
print(true_cate_ranking)

[5, 2, 9, 0, 3, 8, 4, 6, 7, 1]
[2, 5, 9, 0, 4, 7, 8, 3, 1, 6]
[4, 2, 5, 0, 7, 9, 3, 8, 1, 6]


In [21]:
len(true_cate_ranking)

10

In [22]:
true_cate_ranking

[4, 2, 5, 0, 7, 9, 3, 8, 1, 6]

In [23]:
eig_results

{'EIG_obs_from_samples': [],
 'EIG_caus_from_samples': [],
 'EIG_obs_closed_form': [29.916125773741577,
  20.028918055004876,
  43.83158189782415,
  28.40350575807546,
  24.330712490793115,
  44.27452984791596,
  23.658065377554745,
  21.655937347254575,
  27.405115967098048,
  36.14573118673479],
 'EIG_caus_closed_form': [13.470743289849512,
  7.536381961095582,
  23.944506030739092,
  7.820928650295944,
  12.429043910131147,
  23.655109752920005,
  5.3690216866021565,
  10.433787061404736,
  9.84670261588267,
  16.290992448995002],
 'EIG_obs_bart': [],
 'EIG_caus_bart': []}

In [24]:
k =[1,3,5]
top_n = None

In [25]:
# correlation_with_true_rankings={'tau':[],'rho':[], \
#       'recall_at_k':[], 'mean average precision': [], \
#       'ndcg': [], 'rank corr eig': [], 'mean reciprocal rank': []}

correlation_with_true_rankings={}

for val in k:
    correlation_with_true_rankings['precision_at_'+str(val)] = []
compare_to_ground_truth(correlation_with_true_rankings, true_cate_ranking, obs_eig_ranking_closed_form, merged_mse=merged_mse, top_n = top_n, k = k)
compare_to_ground_truth(correlation_with_true_rankings, true_cate_ranking, caus_eig_ranking_closed_form,merged_mse=merged_mse, top_n = top_n, k = k)

# compare_to_ground_truth(correlation_with_true_rankings, true_cate_ranking, obs_eig_ranking_from_samples, top_n = top_n, k = k)
# compare_to_ground_truth(correlation_with_true_rankings, true_cate_ranking, caus_eig_ranking_from_samples, top_n = top_n, k = k)


{'precision_at_1': [0.0, 0.0],
 'precision_at_3': [0.6666666666666666, 0.6666666666666666],
 'precision_at_5': [0.6, 0.8],
 'tau': [0.4222222222222222, 0.6888888888888888],
 'rho': [0.5272727272727272, 0.8181818181818182]}

### Baselines

In [26]:
import copy
from sklearn.linear_model import LogisticRegression

### random ranking
random_ranking = np.random.choice(np.arange(1, number_of_candidate_sites+1), size=number_of_candidate_sites, replace=False)


### ranking by sample size
sample_size_order = sorted(candidate_sites.keys(), key=lambda key: -candidate_sites[key].shape[0])


### ranking by similarity of covariate distribution
mean_vector_host = XandT_host.iloc[:,:causal_param_first_index].mean()
cov_matrix_host = XandT_host.iloc[:,:causal_param_first_index].cov()
mvn = multivariate_normal(mean=mean_vector_host, cov=cov_matrix_host, allow_singular=1)
# get log likelihood of candidate sites
log_likelihood_list=[]
for i, candidate in candidate_sites.items():
    log_likelihoods=mvn.logpdf(candidate.iloc[:,:causal_param_first_index].values)
    log_likelihood_list.append(np.mean(log_likelihoods))

similarity_cov_distrib_ranking= sorted(range(len(log_likelihood_list)), key=lambda i: log_likelihood_list[i], reverse=True)

### ranking by similarity of propensity scores
# we fit a propensity score model at target site and store logloss
# for each site: we fit the model further on the cand site and compute log
# nd assess the loss. Sites associated with loss values with higher discrepancy from the host should have distinct 
#treatment allocation scheme, and thus be a better fit. 

ps_model = LogisticRegression(fit_intercept=True)
ps_model.fit(XandT_host.iloc[:,:causal_param_first_index], XandT_host['T'])
t_host_pred = ps_model.predict(XandT_host.iloc[:,:causal_param_first_index])
mse_host = mean_squared_error(t_host_pred, XandT_host['T'])
mse_diff_list = []


for i, candidate in candidate_sites.items():
    # ps_model_copy= copy.deepcopy(ps_model)
    # ps_model_copy.fit(candidate.iloc[:,:causal_param_first_index], candidate['T'])
    t_cand_pred = ps_model.predict(candidate.iloc[:,:causal_param_first_index]) # predict on host!
    mse_cand = abs(mean_squared_error(t_cand_pred, candidate['T']) - mse_host)
    mse_diff_list.append(mse_cand)

similarity_pscore_ranking = sorted(range(len(mse_diff_list)), key=lambda i: mse_diff_list[i], reverse=True) 
# the more diff in pscore the better so reverse=True


print(random_ranking)
print(sample_size_order)
print(similarity_cov_distrib_ranking)
print(similarity_pscore_ranking)

[ 4  2  9  8  1  5  6 10  3  7]
[10, 4, 3, 9, 6, 7, 1, 2, 5, 8]
[4, 8, 1, 7, 9, 3, 0, 2, 5, 6]
[6, 2, 5, 4, 7, 3, 8, 0, 1, 9]


 /Users/jake/Python/causal_prospective_merge/.conda/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning:lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [27]:
sorted(candidate_sites.keys(), key=lambda key: -candidate_sites[key].shape[0]).index(4)

1

In [28]:
len(candidate_sites.keys())

10

In [29]:
len(similarity_pscore_ranking)

10

In [30]:
random_ranking

array([ 4,  2,  9,  8,  1,  5,  6, 10,  3,  7])

In [31]:
list(sample_size_order).index(4)

1

In [32]:
compare_to_ground_truth(correlation_with_true_rankings, true_cate_ranking, list(random_ranking),merged_mse=merged_mse, top_n = top_n, k = k)
compare_to_ground_truth(correlation_with_true_rankings, true_cate_ranking, list(sample_size_order),merged_mse=merged_mse, top_n = top_n, k = k)
compare_to_ground_truth(correlation_with_true_rankings, true_cate_ranking, list(similarity_cov_distrib_ranking),merged_mse=merged_mse, top_n = top_n, k = k)
compare_to_ground_truth(correlation_with_true_rankings, true_cate_ranking, list(similarity_pscore_ranking),merged_mse=merged_mse, top_n = top_n, k = k)

{'precision_at_1': [0.0, 0.0, 1.0, 0.0, 1.0, 0.0],
 'precision_at_3': [0.6666666666666666,
  0.6666666666666666,
  0.6666666666666666,
  0.3333333333333333,
  0.3333333333333333,
  0.6666666666666666],
 'precision_at_5': [0.6, 0.8, 0.4, 0.2, 0.4, 0.8],
 'tau': [0.4222222222222222,
  0.6888888888888888,
  -0.24444444444444444,
  -0.022222222222222223,
  -0.022222222222222223,
  0.24444444444444444],
 'rho': [0.5272727272727272,
  0.8181818181818182,
  -0.24848484848484845,
  -0.12727272727272726,
  0.05454545454545454,
  0.24848484848484845]}

### Show results

In [33]:
correlation_with_true_rankings= pd.DataFrame.from_dict(correlation_with_true_rankings)
correlation_with_true_rankings.index = ['obs_closed_form', 'caus_closed_form', 'random', 'sample size', 'similarity_cov_distrib_ranking', 'similarity_pscore_ranking size'] #, 'obs_from_samples', 'caus_from_samples']
correlation_with_true_rankings

,precision_at_1,precision_at_3,precision_at_5,tau,rho
obs_closed_form,0.0,0.666667,0.6,0.422222,0.527273
caus_closed_form,0.0,0.666667,0.8,0.688889,0.818182
random,1.0,0.666667,0.4,-0.244444,-0.248485
sample size,0.0,0.333333,0.2,-0.022222,-0.127273
similarity_cov_distrib_ranking,1.0,0.333333,0.4,-0.022222,0.054545
similarity_pscore_ranking size,0.0,0.666667,0.8,0.244444,0.248485


In [34]:
kendalltau(rank ,(merged_mse))

NameError: name 'rank' is not defined

In [ ]:
kendalltau(true_cate_ranking, merged_mse)

SignificanceResult(statistic=0.0717948717948718, pvalue=0.5141058004345272)

In [ ]:
spearmanr(rank, (merged_mse))

SignificanceResult(statistic=1.0, pvalue=0.0)

In [ ]:
###### below is version where ground truth is wrt merged dataset

# for i, candidate in merged_datasets.items():

#     XandT_merged = candidate.drop(columns=["Y"])
#     Y_merged = candidate['Y']

#     learner = LinearRegression(fit_intercept=False)
#     learner.fit(y=Y_merged, X=XandT_merged) # we fit on merged datasets

#     true_cate = data_with_groundtruth.loc[XandT_merged.index, 'ite']

#     X_zero = XandT_merged.copy() # we predict on host with T=0 and T=1
#     X_zero.iloc[:,causal_param_first_index:] = 0

#     X_one = XandT_merged.copy()
#     X_one.iloc[:,causal_param_first_index:] = XandT_merged.iloc[:,:causal_param_first_index]

#     pred_cate = learner.predict(X_one)-learner.predict(X_zero)

#     merged_mse.append(mean_squared_error(true_cate, pred_cate))

## bart stuff

In [ ]:
# X_host, T_host, Y_host = host.drop(columns=['T','Y']).values, host['T'].values.astype(np.int32), host['Y'].values

# prior_hyperparameters = {'sigma_0_sq':1, 'p_categorical_pr':0, 'p_categorical_trt':0 }
# predictive_model_parameters={"num_trees_pr":200,"num_trees_trt":100}
# conditional_model_param={"num_trees_pr":200}


# for i, candidate in candidate_sites.items():

#     print("from samples "+str(i))
#     X_cand, T_cand = candidate.drop(columns=['Y','T']).values, candidate['T'].values.astype(np.int32)

#     bcf = BayesianCausalForest(
#         prior_hyperparameters,
#         predictive_model_parameters=predictive_model_parameters,
#         conditional_model_param=conditional_model_param)
#     bcf.store_train_data(X=X_host, T=T_host, Y=Y_host)
    
#     joint_eig = bcf.joint_EIG_calc(X_cand, T_cand, sampling_parameters)

#     results["EIG_obs_bart"].append(joint_eig["Obs EIG"])
#     results["EIG_caus_bart"].append(joint_eig["Causal EIG"])